In [56]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import yfinance as yf

In [36]:
def gen_portfolio(tickers, start, interval):
    index = None
    return_rates = {}
    for tkr in tickers:
        if interval == '1y':
            data = yf.Ticker(tkr).history(start=start, interval='1mo').iloc[::12]
        else:
            data = yf.Ticker(tkr).history(start=start, interval=interval)
        data = data.Close.pct_change().dropna()
        index = data.index
        return_rates[tkr] = data.to_numpy()
    return pd.DataFrame(return_rates, index=index)

In [40]:
pf_indices = gen_portfolio(['^GSPC', 'AGG', '^SPGSCI'], '2004-01-1', '1y')
pf_tech = gen_portfolio(['AAPL', 'MSFT', 'AMZN'], '2000-01-01', '1y')

In [24]:
RISK_FREE = 0.05

def shrp(return_rates, w):
    R = np.matmul(return_rates.to_numpy(), w)
    return (R.mean()-0.05)/R.std()

In [54]:
def numerical_gradient(w, i, h):
    w_new = w.copy()
    w_new[i] = w[i]+h
    return w_new/w_new.sum()

def gradient_ascent(return_rates, epochs = 10000, learning_rate = 0.001):
    w = np.array([1/return_rates.shape[1] for _ in range(return_rates.shape[1])])
    h = h = 0.0000001
    max_score = 0
    max_w = []

    for _ in range(epochs):
        grad = np.zeros(len(w))
        for i in range(len(w)):
            w_l = numerical_gradient(w, i, -h)
            w_r = numerical_gradient(w, i, h)
            grad[i] = (shrp(return_rates, w_r)-shrp(return_rates, w_l))/(2*h)
        w += learning_rate*grad
        w /= w.sum()
        w[w < 0] = 0
        w /= w.sum()
        score = shrp(return_rates, w)
        if score > max_score:
            max_w = w
            max_score = score

    print(max_score)
    return max_w

In [55]:
print(gradient_ascent(pf_indices))
print(gradient_ascent(pf_tech))

0.25190443568076165
[1. 0. 0.]
0.6587758769633942
[0.489355 0.       0.510645]


In [63]:
def optimize_shrp(return_rates):
    w0 = [1/return_rates.shape[1] for _ in range(return_rates.shape[1])]
    bounds = [(0, 1) for _ in range(len(w0))]
    result = minimize(lambda w: -shrp(return_rates, w), w0, bounds=bounds, constraints={'type': 'eq', 'fun': lambda x: sum(x)-1})
    print(-result.fun)
    return result.x

In [64]:
print(optimize_shrp(pf_indices))
print(optimize_shrp(pf_tech))

0.25190443568076165
[1. 0. 0.]
0.6587758971199779
[0.4891966 0.        0.5108034]
